In [1]:
import os

In [4]:
pip install flask GitPython python-dotenv openai==0.28 tiktoken chromadb==0.4.4 langchain==0.0.249

     |████████████████████████████████| 76 kB 5.8 MB/s  eta 0:00:01
     |████████████████████████████████| 1.8 MB 38.5 MB/s eta 0:00:01
     |████████████████████████████████| 402 kB 60.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 50.5 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 54.1 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 777 kB 59.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 54.8 MB/s eta 0:00:01
Processing /config/.cache/pip/wheels/54/4a/f8/2803c6041841502d0d21fb2a62d401d14716dfeb2261a3a70b/PyPika-0.48.9-py2.py3-none-any.whl
     |████████████████████████████████| 3.2 MB 59.0 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 57.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.4 MB 56.9 MB/s eta 0:00:01
     |████████████████████████████████| 6.

In [5]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain